In [ ]:
# Import the functions
import pandas as pd
from labelgenius import classification_CLIP_0_shot, finetune_CLIP, classification_CLIP_finetuned, auto_verification

Demo 1: Single-Category Classification using N24News Dataset
-------------------------------------------------------------

This demo shows how to classify a single news article into one of 24 category
using the N24News dataset. Each article in the dataset includes both textual
and visual information.

Source: https://aclanthology.org/2022.lrec-1.729/


Each article contains the following fields:
- 'section': Ground truth label (one of 24 category)
- 'headline': Title of the article
- 'abstract': Short summary of the article
- 'article': Full text content
- 'article_url': Link to the original article
- 'image': Encoded image or metadata (optional)
- 'caption': Image caption
- 'image_id': Unique image identifier
- 'image_path': Path to the associated image (e.g., 'N24News/imgs_200_sample1/12345.jpg')
- 'article_id': Unique article identifier

Image file: Multimodal_image

Example category (See prompt_D1 for the complete category):
------------------------
1. Health
2. Science
3. Television
...
24. Global Business

Reference:
----------
Wang, Z., Shan, X., Zhang, X., & Yang, J. (2022).
N24News: A New Dataset for Multimodal News Classification.
In *Proceedings of the Thirteenth Language Resources and Evaluation Conference* (pp. 6768–6775). LREC.


### Demo 1a: Single-Category Text Classification

**Datasets:**
- `D1_1.csv`: Used for initial labeling and fine-tuning.
- `D1_2.csv`: Used for testing the fine-tuned model's performance.


In [ ]:
# Define the descriptive prompt for each category
prompt_D1_CLIP = [
    "a news article about health, including medical news, public health issues, fitness, mental health, and wellness.",
    "a news article about science, covering scientific discoveries, research studies, space exploration, and innovations.",
    "a news article about television, featuring TV shows, series reviews, industry news, and streaming platforms.",
    "a news article about travel, focusing on tourism, destinations, travel guides, airlines, and vacation trends.",
    "a news article about movies, including film industry news, reviews, box office reports, and upcoming releases.",
    "a news article about dance, covering ballet, contemporary styles, street dance, performances, and dance competitions.",
    "a news article about real estate, highlighting housing market trends, property sales, architecture, and urban planning.",
    "a news article about the economy, featuring macroeconomics, inflation, stock markets, GDP growth, and financial policies.",
    "a news article about sports, covering professional sports, competitions, athlete news, and game Demo_results.",
    "a news article about theater, featuring plays, Broadway shows, live performances, and stage production reviews.",
    "a news article about opinion pieces, including editorials, analysis, and expert commentaries.",
    "a news article about music, covering albums, artists, concerts, festivals, and industry trends.",
    "a news article about books, featuring literature, bestsellers, author interviews, and book reviews.",
    "a news article about art and design, showcasing fine arts, visual arts, museums, exhibitions, and design trends.",
    "a news article about style, including fashion trends, beauty, personal style, and cultural aesthetics.",
    "a news article about media, covering journalism, publishing, digital media, and mass communication.",
    "a news article about food, featuring restaurants, cooking, recipes, culinary trends, and food culture.",
    "a news article about well-being, focusing on lifestyle, personal development, mental well-being, and self-care.",
    "a news article about fashion, covering clothing, designers, fashion weeks, and industry insights.",
    "a news article about technology, featuring AI, gadgets, software, cybersecurity, and tech innovations.",
    "a news article about personal finance, including investing, budgeting, and financial planning.",
    "a news article about education, featuring schools, universities, learning methods, and education policies.",
    "a news article about automobiles, covering car industry news, electric vehicles, reviews, and trends.",
    "a news article about global business, featuring international trade, corporations, mergers, and global markets."
]

# Define the list of 24 category labels
category_D1_CLIP = [
    "1", "2", "3", "4", "5", "6",
    "7", "8", "9", "10", "11", "12",
    "13", "14", "15", "16", "17", "18",
    "19", "20", "21", "22", "23", "24"
]

## CLIP: local labeling

In [ ]:
D1a_CLIP_inital_labeling = classification_CLIP_0_shot(
    text_path="Demo_data/D1_1.csv",
    mode="text",
    prompt=prompt_D1_CLIP,
    text_column=["headline", "abstract"],
    predict_column="D1a_CLIP_inital_labeling",
)


D1a_CLIP_inital_labeling.to_csv("Demo_result/D1a_CLIP_inital_labeling.csv", index=False)

In [ ]:
## Check the accuracy: CLIP initial labeling
auto_verification(
    D1a_CLIP_inital_labeling,
    predicted_cols="D1a_CLIP_inital_labeling",
    true_cols="section_numeric",
    category=category_D1_CLIP
)

### finetune: CLIP

In [ ]:
# finetune CLIP
finetune_CLIP(
    mode="text",
    text_path="Demo_data/D1_1.csv",
    text_column=["headline", "abstract"],
    true_label="section_numeric",
    model_name="Demo_finetuned_CLIP/D1a_CLIP_model_finetuned.pth",
    num_epochs=20,
    batch_size=8,
    learning_rate=1e-5,
)


In [ ]:
# Classify with a fine‑tuned model
D1a_CLIP_finetuned = classification_CLIP_finetuned(
    mode="text",
    text_path="Demo_data/D1_2.csv",
    text_column=["headline", "abstract"],
    model_name="Demo_finetuned_CLIP/D1a_CLIP_model_finetuned.pth",
    predict_column="D1a_CLIP_finetuned",
)


D1a_CLIP_finetuned.to_csv("Demo_result/D1a_CLIP_finetuned.csv", index=False)

In [ ]:
## Check the accuracy: CLIP after finetune
auto_verification(
    D1a_CLIP_finetuned,
    predicted_cols="D1a_CLIP_finetuned",
    true_cols="section_numeric",
    category=category_D1_CLIP
)

# Demo 1b: Classify single-category image data

**Datasets:**

- `D1_1.csv`: Used for initial labeling and fine-tuning.
- `imgs_40_1`: Used for initial labeling and fine-tuning.

- `D1_2.csv`: Used for testing the fine-tuned model's performance.
- `imgs_40_2`: Used for testing the fine-tuned model's performance.


### CSV File Requirements

Each CSV file must include at minimum:

- `image_id`  
  – The base filename (without extension) of each image in the corresponding folder.

## CLIP: local labeling

In [ ]:
D1b_CLIP_inital_labeling = classification_CLIP_0_shot(
    text_path="Demo_data/D1_1.csv",
    img_dir="Demo_data/D1_imgs_1",
    mode="image",
    prompt=prompt_D1_CLIP,
    predict_column="D1b_CLIP_inital_labeling",
)


D1b_CLIP_inital_labeling.to_csv("Demo_result/D1b_CLIP_inital_labeling.csv", index=False)

In [ ]:
D1b_CLIP_inital_labeling

In [ ]:
## Check the accuracy: CLIP initial labeling
auto_verification(
    D1b_CLIP_inital_labeling,
    predicted_cols="D1b_CLIP_inital_labeling",
    true_cols="section_numeric",
    category=category_D1_CLIP
);

### finetune: CLIP

In [ ]:
finetune_CLIP(
    mode="image",
    text_path="Demo_result/D1b_CLIP_inital_labeling.csv",
    img_dir="Demo_data/D1_imgs_1",
    true_label="section_numeric",
    model_name="Demo_finetuned_CLIP/D1a_CLIP_model_finetuned.pth",
    num_epochs=20,
    batch_size=8,
    learning_rate=1e-5,
)


In [ ]:
# Classify with a fine‑tuned model
D1b_CLIP_finetuned = classification_CLIP_finetuned(
    mode="image",
    text_path="Demo_data/D1_2.csv",
    img_dir="Demo_data/D1_imgs_2",
    model_name="Demo_finetuned_CLIP/D1a_CLIP_model_finetuned.pth",
    predict_column="D1b_CLIP_finetuned",
)

D1b_CLIP_finetuned.to_csv("Demo_result/D1b_CLIP_finetuned.csv", index=False)

In [ ]:
## Check the accuracy: CLIP after finetune
# Merge the Demo_result of the image with the human label
auto_verification(
    D1b_CLIP_finetuned,
    predicted_cols="D1b_CLIP_finetuned",
    true_cols="section_numeric",
    category=category_D1_CLIP
)

# Demo 1c: Classify single-category text + image data

**Datasets:**
- `D1_1.csv`: Used for initial labeling and fine-tuning.
- `D1_imgs_1`: Used for initial labeling and fine-tuning.

- `D1_2.csv`: Used for testing the fine-tuned model's performance.
- `D1_imgs_2`: Used for testing the fine-tuned model's performance.


The text dataset should contain a column `image_path` to map the images for each row.

## CLIP: local labeling

In [ ]:
D1c_CLIP_inital_labeling = classification_CLIP_0_shot(
    text_path="Demo_data/D1_1.csv",
    img_dir="Demo_data/D1_imgs_1",
    mode="both",
    prompt=prompt_D1_CLIP,
    text_column=["headline", "abstract"],
    predict_column="D1c_CLIP_inital_labeling",
)

D1c_CLIP_inital_labeling.to_csv("Demo_result/D1c_CLIP_inital_labeling.csv", index=False)

In [ ]:
## Check the accuracy: CLIP initial labeling
auto_verification(
    D1c_CLIP_inital_labeling,
    predicted_cols="D1c_CLIP_inital_labeling",
    true_cols="section_numeric",
    category=category_D1_CLIP
);

### finetune: CLIP

In [ ]:
# finetune CLIP
finetune_CLIP(
    mode="both",
    text_path="Demo_data/D1_1.csv",
    text_column=["headline", "abstract"],
    img_dir="Demo_data/D1_imgs_1",
    true_label="section_numeric",
    model_name="Demo_finetuned_CLIP/D1c_CLIP_model_finetuned.pth",
    num_epochs=20,
    batch_size=8,
    learning_rate=1e-5,
)


In [ ]:
# Classify with a fine‑tuned model
D1c_CLIP_finetuned = classification_CLIP_finetuned(
    mode="both",
    text_path="Demo_data/D1_2.csv",
    text_column=["headline", "abstract"],
    img_dir="Demo_data/D1_imgs_2",
    model_name="Demo_finetuned_CLIP/D1c_CLIP_model_finetuned.pth",
    predict_column="D1c_CLIP_finetuned",
    
)
D1c_CLIP_finetuned.to_csv("Demo_result/D1c_CLIP_finetuned.csv", index=False)

In [ ]:
D1c_CLIP_finetuned

In [ ]:
## Check the accuracy: CLIP after finetune
auto_verification(
    D1c_CLIP_finetuned,
    predicted_cols="D1c_CLIP_finetuned",
    true_cols="section_numeric",
    category=category_D1_CLIP
)